# Dados abertos CAPES - Modelagem para painel do GID

In [ ]:
#Importando bibliotecas necessárias:
import os
import time
import re
import ssl
import requests
from urllib.parse import urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import operator

In [9]:
#Diretórios usados para armazenar os dados (baixados e processados)
dirs = {
    'download_dir': 'capes_csv_files',
    'ufrj_dir': 'ufrj_data',
    'discentes': 'discentes',
    'docentes': 'docentes',
    'programas': 'programas',
    'cursos':  'programas',
    'producao': 'producao',
    'producao_detalhe': 'producao',
    'producao_autor': 'producao',
    'projetos': 'projetos',
    'membros': 'projetos',
    'financiadores': 'financiadores',
    'btd': 'btd',
}

## Download dos dados abertos da CAPES

In [10]:
# Configurações do download
api_url = "https://dadosabertos.capes.gov.br/api/3/action/package_search"
organization = "diretoria-de-avaliacao"
output_dir = dirs.get('download_dir', 'capes_csv_files')
timeout_seconds = 30
max_retries = 3


prefix_substring_dirs = {
    "ddi-br-capes-colsucup-": {
        "projeto-financiador": dirs.get("financiadores", "financiadores"),
        "projeto": dirs.get("projetos", "projetos"),
    }, 
    "br-capes-colsucup-": {
        "prod": dirs.get("producao", "producao"),
        "producao": dirs.get("producao", "producao"),
        "projeto": dirs.get("projetos", "projetos"),
        "membro": dirs.get("projetos", "projetos"),
        "prog": dirs.get("programas", "programas"),
        "curso": dirs.get("cursos", "cursos"),
		"discentes": dirs.get("discentes", "discentes"),
		"docente": dirs.get("docentes", "docentes"),
        "financiador": dirs.get("financiadores", "financiadores"),
    },
    "br-capes-col-": {
        "proj": dirs.get("projetos", "projetos"),
        "producao": dirs.get("producao", "producao"),
        "prod": dirs.get("producao", "producao"),
    },
	"br-colsucup-": {
		"prod": dirs.get("producao", "producao"),
	},
    "br-capes-btd-": {
        "": dirs.get("btd", "btd")
    },
} # Mapeia prefix+substring para pastas (diretórios)

# Sessão com retry
session = requests.Session()
retry = Retry(total=max_retries, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Contexto SSL customizado (caso precise ignorar erros SSL)
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Criar diretório base
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(filename):
    """Remove caracteres inválidos para nomes de arquivo"""
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

def get_with_retry(url):
    """Faz uma requisição HTTP com retry e fallback para SSL desabilitado ou HTTP"""
    for attempt in range(max_retries):
        try:
            print(f"Tentativa {attempt + 1} para {url}")
            try:
                response = session.get(url, timeout=timeout_seconds)
                response.raise_for_status()
                return response
            except requests.exceptions.SSLError:
                print("Falha SSL, tentando com verificação desativada...")
                response = session.get(url, timeout=timeout_seconds, verify=False)
                response.raise_for_status()
                return response
        except requests.exceptions.RequestException as e:
            print(f"Erro na tentativa {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
    if url.startswith('https://'):
        http_url = url.replace('https://', 'http://', 1)
        print(f"Tentando fallback para HTTP: {http_url}")
        try:
            response = session.get(http_url, timeout=timeout_seconds)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Falha no fallback HTTP: {e}")
    return None

def get_all_csv_links_from_ckan():
    """Consulta a API CKAN e retorna todos os arquivos CSV da organização"""
    print("Consultando a API CKAN da CAPES...")
    params = {
        "fq": f"organization:{organization}",
        "rows": 1000
    }
    try:
        response = session.get(api_url, params=params, timeout=timeout_seconds)
        response.raise_for_status()
        results = response.json()["result"]["results"]
        csv_links = []
        for dataset in results:
            for resource in dataset.get("resources", []):
                if resource.get("format", "").lower() == "csv":
                    url = resource.get("url")
                    if url:
                        csv_links.append(url)
        return sorted(set(csv_links))
    except Exception as e:
        print(f"Erro ao consultar CKAN: {e}")
        return []

def detect_subfolder(filename):
    """Detecta subpasta com base nas substrings do nome do arquivo"""
    name = filename.lower()
    for prefix, substrings in prefix_substring_dirs.items():
        for substr, folder in substrings.items():
            target = prefix + substr
            if target in name:
                return folder
    return "outros"


def download_csv_files(links):
    """Baixa todos os arquivos CSV e organiza em subpastas por tipo"""
    total = len(links)
    print(f"\nIniciando download de {total} arquivos...")

    for i, url in enumerate(links, 1):
        try:
            raw_filename = url.split('/')[-1].split('?')[0]
            filename = sanitize_filename(raw_filename)
            subfolder = detect_subfolder(filename)

            subdir_path = os.path.join(output_dir, subfolder)
            os.makedirs(subdir_path, exist_ok=True)

            filepath = os.path.join(subdir_path, filename)

            if os.path.exists(filepath):
                print(f"[{i}/{total}] Já existe: {subfolder}/{filename}")
                continue

            print(f"[{i}/{total}] Baixando: {filename} para {subfolder}/")

            start_time = time.time()
            response = get_with_retry(url)
            if not response:
                print(f"Falha ao baixar {url}")
                continue

            with open(filepath, 'wb') as f:
                f.write(response.content)

            size_mb = os.path.getsize(filepath) / (1024 * 1024)
            print(f"Salvo como {subfolder}/{filename} ({size_mb:.2f} MB) em {time.time() - start_time:.2f}s")

        except Exception as e:
            print(f"Erro ao baixar {url}: {e}")

In [12]:
#Código para download dos dados abertos - Processo potencialmente demorado
print("Iniciando processo para baixar todos os CSVs da Diretoria de Avaliação...")
start_time = time.time()
try:
    csv_links = get_all_csv_links_from_ckan()
    print(f"\n{len(csv_links)} arquivos CSV encontrados:")
    for i, link in enumerate(csv_links[:10], 1):
        print(f"{i}. {link.split('/')[-1].split('?')[0]}")
    if len(csv_links) > 10:
        print(f"... mais {len(csv_links) - 10} arquivos")
    if csv_links:
        download_csv_files(csv_links)
    print(f"\nConcluído em {time.time() - start_time:.2f}s")
except KeyboardInterrupt:
    print("\nProcesso interrompido pelo usuário.")
except Exception as e:
    print(f"Erro inesperado: {e}")

Iniciando processo para baixar todos os CSVs da Diretoria de Avaliação...
Consultando a API CKAN da CAPES...

429 arquivos CSV encontrados:
1. br-capes-colsucup-producao-2013a2016-2017-11-01-tecnica-cursocdu.csv
2. br-capes-colsucup-producao-2013a2016-2017-11-01-tecnica-progrtv.csv
3. br-capes-colsucup-producao-2013a2016-2017-11-01-bibliografica-tradu.csv
4. br-capes-colsucup-producao-2013a2016-2017-11-01-artistica-visual.csv
5. br-capes-colsucup-producao-2013a2016-2017-11-01-tecnica-destec.csv
6. br-capes-colsucup-producao-2013a2016-2017-11-01-tecnica-patente.csv
7. br-capes-colsucup-producao-2013a2016-2017-11-01-artistica-outra.csv
8. br-capes-colsucup-producao-2013a2016-2017-11-01-bibliografica-livro.csv
9. br-capes-colsucup-producao-2013a2016-2017-11-01-tecnica-dprodu.csv
10. br-capes-colsucup-producao-2013a2016-2017-11-01-tecnica-deapli.csv
... mais 419 arquivos

Iniciando download de 429 arquivos...
[1/429] Já existe: producao/br-capes-colsucup-producao-2013a2016-2017-11-01-tecni

## Extraindo apenas os dados da UFRJ dos dados abertos CAPES - 2013 em diante

In [ ]:
#Função para selecionar documentos csv via expressões regulares e após um dado ano (e.g. 2013)
#A identificação do ano através do nome de arquivo desta função está adaptada à padronização de nomes dos arquivos da CAPES
#Valores de anos que iniciam quadriênios (2013, 2017, 2021) funcionarão normalmente. Outros anos poderão apresentar problemas
def filtrar_csvs_por_diretorio_regex_e_ano(
    caminho_do_diretorio,
    padroes_regex=None,
    ano_minimo=None, 
    busca_recursiva=False
):
    """
    Lista arquivos CSV em um diretório e os filtra com base em:
    1. Padrões de expressão regular (regex)
    2. Um ano mínimo encontrado no nome do arquivo.

    Args:
        caminho_do_diretorio (str): O caminho para o diretório base onde os CSVs estão.
        padroes_regex (list, optional): Uma lista de strings de expressões regulares.
                                        Arquivos devem corresponder a *qualquer um* desses padrões.
                                        Default é None (não aplica filtro regex).
        ano_minimo (int, optional): O ano mínimo (4 dígitos) para filtrar.
                                    Captura o primeiro grupo de 4 dígitos encontrado no nome do arquivo.
                                    Default é None (não aplica filtro de ano).
        busca_recursiva (bool, optional): Se True, a função buscará CSVs em subpastas também.
                                          Default é False.

    Returns:
        list: Uma lista de caminhos completos para os arquivos CSV que atendem a todos os critérios.

    Raises:
        ValueError: Se 'caminho_do_diretorio' não for encontrado ou se nenhum filtro for especificado.
    """
    if not os.path.exists(caminho_do_diretorio):
        print(f"Erro: O diretório '{caminho_do_diretorio}' não foi encontrado.")
        return []

    if not padroes_regex and ano_minimo is None:
        print("Aviso: Nenhum filtro (regex ou ano mínimo) foi especificado. Retornando todos os CSVs.")

    todos_csvs_encontrados = []

    # 1. Obter todos os arquivos CSV do diretório (e subpastas, se recursivo)
    if busca_recursiva:
        for root, _, files in os.walk(caminho_do_diretorio):
            for nome_arquivo in files:
                if nome_arquivo.lower().endswith(".csv"):
                    todos_csvs_encontrados.append(os.path.join(root, nome_arquivo))
    else:
        for nome_arquivo in os.listdir(caminho_do_diretorio):
            caminho_completo = os.path.join(caminho_do_diretorio, nome_arquivo)
            if os.path.isfile(caminho_completo) and nome_arquivo.lower().endswith(".csv"):
                todos_csvs_encontrados.append(caminho_completo)

    arquivos_filtrados_parcial = todos_csvs_encontrados

    # 2. Aplicar filtro por padrões Regex (se fornecidos)
    if padroes_regex:
        final_regex_filter = []
        regexes_compilados = [re.compile(p, re.IGNORECASE) for p in padroes_regex]

        for caminho_completo in arquivos_filtrados_parcial:
            nome_base = os.path.basename(caminho_completo)
            for regex in regexes_compilados:
                if regex.search(nome_base):
                    final_regex_filter.append(caminho_completo)
                    break
        arquivos_filtrados_parcial = final_regex_filter

    # 3. Aplicar filtro por ano mínimo (se fornecido)
    if ano_minimo is not None:
        final_year_filter = []
        # Regex para qualquer sequência de 4 dígitos - adaptado à padronização CAPES
        # O padrão (\d{4}) captura o ano. Valores de anos que iniciam quadriênios (2013, 2017, 2021)
        # funcionarão normalmente. Outros anos poderão apresentar problemas se o formato 'discentes-AAAA'
        # não for o primeiro e único lugar onde o ano pode estar.
        padrao_ano_capes = re.compile(r'(\d{4})') 

        for caminho_completo in arquivos_filtrados_parcial:
            nome_base = os.path.basename(caminho_completo)
            match = padrao_ano_capes.search(nome_base)
            
            if match:
                ano_str = match.group(1)
                try:
                    ano_int = int(ano_str)
                    if ano_int >= ano_minimo:
                        final_year_filter.append(caminho_completo)
                except ValueError:
                    print(f"Aviso: Não foi possível converter '{ano_str}' em ano inteiro para '{nome_base}'. Ignorando.")
        arquivos_filtrados_parcial = final_year_filter

    return arquivos_filtrados_parcial

In [94]:
#Função para obter caminhos utilizando o dicionário de diretórios definido no começo deste documento
def obter_caminho_completo(basedir_key, subdir_key, dirs=dirs):
    """
    Concatena os caminhos correspondentes a 'basedir_key' e 'subdir_key'
    do dicionário 'dirs' para formar um caminho completo.

    Args:
        basedir_key (str): A chave do diretório base em 'dirs'.
        subdir_key (str): A chave do subdiretório em 'dirs'.
        dirs (dict): Dicionário com o nome dos diretórios. Default: dirs.

    Returns:
        str: O caminho completo concatenado.

    Raises:
        ValueError: Se 'basedir_key' ou 'subdir_key' não forem chaves válidas em 'dirs'.
    """
    chaves_disponiveis = list(dirs.keys())

    if basedir_key not in dirs:
        raise ValueError(
            f"Erro: Chave '{basedir_key}' não encontrada em 'dirs' para basedir. "
            f"Chaves disponíveis: {chaves_disponiveis}"
        )

    if subdir_key not in dirs:
        raise ValueError(
            f"Erro: Chave '{subdir_key}' não encontrada em 'dirs' para subdir. "
            f"Chaves disponíveis: {chaves_disponiveis}"
        )
    
    # Obtém os valores de diretório do dicionário
    base_path = dirs[basedir_key]
    sub_path = dirs[subdir_key]

    # Concatena os caminhos usando os.path.join para compatibilidade entre sistemas
    caminho_final = os.path.join(base_path, sub_path)
    
    return caminho_final

In [95]:
#Junção das funções 'filtrar_csvs_por_diretorio_regex_e_ano()' e 'obter_caminho_completo()'
#Usada para obter uma lista com os arquivos csv de interesse a serem unificados em uma tabela única
def selecionar_csvs_capes(
    basedir_key,
    subdir_key,
    dirs=dirs,
    padroes_regex=None,
    ano_minimo=2013,
    busca_recursiva=False
):
    """
    Integra as funções para obter o caminho completo do diretório e filtrar arquivos CSV.

    Args:
        dirs (dict): Dicionário com o nome dos diretórios. Default: dirs.
        basedir_key (str): A chave do diretório base em 'dirs' (e.g., 'download_dir', 'ufrj_dir').
        subdir_key (str): A chave do subdiretório em 'dirs' (e.g., 'discentes', 'producao').
        padroes_regex (list, optional): Uma lista de strings de expressões regulares para filtrar nomes de arquivo.
                                        Arquivos devem corresponder a *qualquer um* desses padrões.
                                        Default é None (não aplica filtro regex).
        ano_minimo (int, optional): O ano mínimo (4 dígitos) para filtrar.
                                    Adapta-se ao padrão "discentes-AAAA" dos arquivos da CAPES.
                                    Valores de anos que iniciam quadriênios (2013, 2017, 2021) funcionarão
                                    normalmente. Outros anos poderão apresentar problemas se o formato não se encaixar.
                                    Default é None (não aplica filtro de ano).
        busca_recursiva (bool, optional): Se True, a função buscará CSVs em subpastas do diretório gerado.
                                          Default é False.

    Returns:
        list: Uma lista de caminhos completos para os arquivos CSV que atendem a todos os critérios.

    Raises:
        ValueError: Se 'basedir_key' ou 'subdir_key' forem inválidas, ou se o diretório final não existir.
    """
    try:
        # 1. Obter o caminho completo do diretório usando as chaves
        caminho_do_diretorio_completo = obter_caminho_completo(basedir_key, subdir_key)
        print(f"Buscando arquivos no diretório: {caminho_do_diretorio_completo}")

    except ValueError as e:
        print(f"Erro ao obter caminho do diretório: {e}")
        return []

    # 2. Filtrar os arquivos CSV dentro do diretório gerado
    arquivos_selecionados = filtrar_csvs_por_diretorio_regex_e_ano(
        caminho_do_diretorio=caminho_do_diretorio_completo,
        padroes_regex=padroes_regex,
        ano_minimo=ano_minimo,
        busca_recursiva=busca_recursiva
    )

    return arquivos_selecionados

In [161]:
# Dicionário que mapeia strings para as funções de operador correspondentes.
OPERADORES = {
    'igual': operator.eq,          # ==
    'diferente': operator.ne,      # !=
    'maior': operator.gt,          # >
    'maior_igual': operator.ge,    # >=
    'menor': operator.lt,          # <
    'menor_igual': operator.le,    # <=
    # Adicione outros operadores se precisar, como 'contem' para strings
}

#Função para criação de funções filtro personalizadas a serem aplicadas aos csvs analisados
def criar_funcao_filtro_personalizado(nome_coluna, valor_comparacao, tipo_comparacao='igual'):
    """
    Cria e retorna uma função de filtro personalizada com base em uma coluna,
    um valor de comparação e um tipo de comparação (igual, diferente, maior, etc.).

    Args:
        nome_coluna (str): O nome da coluna a ser verificada.
        valor_comparacao: O valor com o qual comparar o conteúdo da coluna.
                          Para strings com 'igual'/'diferente', a comparação será case-insensitive.
        tipo_comparacao (str): O tipo de comparação a ser realizada.
                               Valores aceitos: 'igual', 'diferente', 'maior', 'maior_igual',
                               'menor', 'menor_igual'. Default é 'igual'.

    Returns:
        function: Uma função (closure) que aceita uma linha do DataFrame (Series)
                  e retorna True se a condição for satisfeita, False caso contrário.

    Raises:
        ValueError: Se o 'tipo_comparacao' não for válido.
    """
    # Valida o tipo de comparação fornecido
    if tipo_comparacao.lower() not in OPERADORES:
        raise ValueError(
            f"Tipo de comparação inválido: '{tipo_comparacao}'. "
            f"Valores aceitos são: {list(OPERADORES.keys())}"
        )
    
    op_funcao = OPERADORES[tipo_comparacao.lower()]

    # Pré-processa o valor de comparação se for string para igualdade/diferença
    # (apenas para consistência se usarmos .upper() para strings)
    valor_comparacao_processado = valor_comparacao
    if isinstance(valor_comparacao, str) and (tipo_comparacao.lower() == 'igual' or tipo_comparacao.lower() == 'diferente'):
        valor_comparacao_processado = valor_comparacao.strip().upper()

    def condicao_filtro_interna(linha):
        """
        Função de filtro real que será passada para processar_csv_otimizado_lista.
        Aplica a comparação dinâmica.
        """
        # Verifica se a coluna existe e não é nula na linha
        if nome_coluna in linha and pd.notna(linha[nome_coluna]):
            valor_na_linha = linha[nome_coluna]

            if isinstance(valor_comparacao_processado, str) and isinstance(valor_na_linha, str) and \
               (tipo_comparacao.lower() == 'igual' or tipo_comparacao.lower() == 'diferente'):
                # Compara strings de forma case-insensitive para 'igual' ou 'diferente'
                return op_funcao(valor_na_linha.strip().upper(), valor_comparacao_processado)
            else:
                # Compara outros tipos de dados ou strings com outros operadores
                try:
                    # Tenta converter o valor da linha para o mesmo tipo do valor de comparação
                    # Isso é útil para comparar números em colunas que podem vir como strings
                    if isinstance(valor_comparacao_processado, (int, float)):
                        valor_na_linha = type(valor_comparacao_processado)(valor_na_linha)
                    return op_funcao(valor_na_linha, valor_comparacao_processado)
                except (ValueError, TypeError):
                    # Lida com casos onde a conversão falha ou tipos são incompatíveis para comparação
                    return False
        return False # Se a coluna não existir, for nula, ou não atender à condição

            # --- LINHA CHAVE: ATRIBUINDO O ATRIBUTO À FUNÇÃO ANINHADA ANTES DE RETORNAR ---
    condicao_filtro_interna.filter_column_name = nome_coluna

    return condicao_filtro_interna

In [158]:
#Definição de filtros e diretório de saída dos arquivos filtrados
filtro_ufrj_sigla = criar_funcao_filtro_personalizado('SG_ENTIDADE_ENSINO', 'UFRJ')
ufrj_dir = dirs.get('ufrj_dir')

In [169]:
#Função para processar e salvar csvs relacionados em um único arquivo
def processar_csvs(
        lista_caminhos_csv, 
        colunas_desejadas, 
        condicao_filtro_funcao=filtro_ufrj_sigla, 
        diretorio_saida=ufrj_dir, 
        nome_arquivo_saida="saida_otimizada_lista.csv", 
        chunk_size=None, 
        remove_filter_col=False
        ):
    """
    Processa uma lista de arquivos CSV, extraindo colunas e linhas específicas de forma otimizada para RAM.

    Args:
        lista_caminhos_csv (list): Uma lista de strings, onde cada string é o caminho completo para um arquivo CSV.
        colunas_desejadas (list): Uma lista de nomes de colunas a serem extraídas.
        condicao_filtro_funcao (function): Uma função que recebe uma linha (como Series do pandas)
                                           e retorna True se a linha deve ser incluída, False caso contrário.
                                           Esta função deve ter um atributo 'filter_column_name'
                                           se criada por 'criar_funcao_filtro_personalizado'.
        diretorio_saida (str): O caminho para o diretório onde o arquivo de saída será salvo.
                               Default é ".", que representa o diretório atual.
        nome_arquivo_saida (str): O nome do arquivo CSV de saída (ex: "meu_arquivo.csv").
                                  Default é "saida_otimizada_lista.csv".
        chunk_size (int, optional): O número de linhas a serem lidas por vez de cada arquivo CSV.
                                   Se None, o arquivo é lido integralmente.
        remove_filter_col (bool): Se True, a coluna usada para o filtro será removida do DataFrame de saída.
                                  Default é False.
    """
    primeiro_arquivo = True
    coluna_filtro = None

    # Constrói o caminho completo do arquivo de saída
    caminho_completo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

    # Garante que o diretório de saída exista
    if diretorio_saida and not os.path.exists(diretorio_saida):
        os.makedirs(diretorio_saida, exist_ok=True)
        print(f"Diretório de saída criado: {diretorio_saida}")

    # Crie uma cópia da lista de colunas desejadas originais para verificar depois.
    colunas_desejadas_originais = list(colunas_desejadas) 

    if hasattr(condicao_filtro_funcao, 'filter_column_name'):
        coluna_filtro = condicao_filtro_funcao.filter_column_name
        # Se a coluna de filtro *não* está nas colunas desejadas originais, adicione-a para leitura.
        if coluna_filtro not in colunas_desejadas: # Esta é a lista que será modificada para leitura
            colunas_desejadas.append(coluna_filtro) # Usa append para adicionar no final, sem reatribuir a lista
            print(f"Adicionando a coluna de filtro '{coluna_filtro}' a 'colunas_desejadas' para otimização da leitura.")
    
    colunas_para_ler = []
    for col in colunas_desejadas: # Processa a lista já potencialmente modificada
        if col not in colunas_para_ler:
            colunas_para_ler.append(col)
    
    colunas_para_salvar = list(colunas_para_ler)

    # NOVO LÓGICA: Só remove a coluna de filtro se remove_filter_col for True E a coluna de filtro
    # NÃO estava na lista original de colunas desejadas.
    if remove_filter_col and \
       coluna_filtro and \
       coluna_filtro in colunas_para_salvar and \
       coluna_filtro not in colunas_desejadas_originais: # <-- AQUI ESTÁ A MUDANÇA
        
        colunas_para_salvar.remove(coluna_filtro)
        print(f"A coluna de filtro '{coluna_filtro}' será removida do arquivo de saída já que 'remove_filter_col=True'.")
    elif remove_filter_col and coluna_filtro and coluna_filtro in colunas_desejadas_originais:
        print(f"A coluna de filtro '{coluna_filtro}' NÃO será removida do arquivo de saída, pois foi explicitamente incluída em 'colunas_desejadas'.")


    for caminho_completo_arquivo_entrada in lista_caminhos_csv:
        if not os.path.exists(caminho_completo_arquivo_entrada):
            print(f"Aviso: Arquivo não encontrado - {caminho_completo_arquivo_entrada}. Pulando...")
            continue
        if not caminho_completo_arquivo_entrada.lower().endswith(".csv"): # Adicionado .lower() para robustez
            print(f"Aviso: Ignorando arquivo não CSV - {caminho_completo_arquivo_entrada}.")
            continue

        print(f"Processando {caminho_completo_arquivo_entrada}...")

        read_csv_args = {
            'sep': ';',
            'encoding': 'latin1',
            'usecols': colunas_para_ler
        }

        if chunk_size is None:
            try:
                df = pd.read_csv(caminho_completo_arquivo_entrada, **read_csv_args)
                df_filtrado = df[df.apply(condicao_filtro_funcao, axis=1)]
                
                df_final = df_filtrado[colunas_para_salvar]

                if primeiro_arquivo:
                    df_final.to_csv(caminho_completo_saida, mode='w', index=False)
                    primeiro_arquivo = False
                else:
                    df_final.to_csv(caminho_completo_saida, mode='a', header=False, index=False)
            except Exception as e:
                print(f"Erro ao ler ou processar o arquivo {caminho_completo_arquivo_entrada} completamente: {e}")
                continue
        else:
            read_csv_args['chunksize'] = chunk_size
            try:
                for chunk in pd.read_csv(caminho_completo_arquivo_entrada, **read_csv_args):
                    df_filtrado = chunk[chunk.apply(condicao_filtro_funcao, axis=1)]
                    
                    df_final = df_filtrado[colunas_para_salvar]

                    if primeiro_arquivo:
                        df_final.to_csv(caminho_completo_saida, mode='w', index=False)
                        primeiro_arquivo = False
                    else:
                        df_final.to_csv(caminho_completo_saida, mode='a', header=False, index=False)
            except Exception as e:
                print(f"Erro ao ler ou processar o arquivo {caminho_completo_arquivo_entrada} em chunks: {e}")
                continue

    print(f"Processamento concluído. Saída salva em {caminho_completo_saida}")

### Discentes 

In [170]:
#Obtendo lista de arquivos csv com ano de referencia = 2013 ou maior (sem filtragem por regex)
discentes_csvs = selecionar_csvs_capes('download_dir', 'discentes')

Buscando arquivos no diretório: capes_csv_files/discentes


In [171]:
discentes_csvs

['capes_csv_files/discentes/br-capes-colsucup-discentes-2021-2025-03-31.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2013-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2022-2025-03-31.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2014-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2020-2023-12-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2016-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2023-2025-03-31.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2018-2023-12-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2015-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2019-2023-12-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2017-2023-12-01.csv']

In [173]:
processar_csvs(discentes_csvs, ['AN_BASE', 'ID_PESSOA', 'CD_AREA_AVALIACAO'], nome_arquivo_saida='discentes.csv')

Adicionando a coluna de filtro 'SG_ENTIDADE_ENSINO' a 'colunas_desejadas' para otimização da leitura.
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2021-2025-03-31.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2013-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2022-2025-03-31.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2014-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2020-2023-12-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2016-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2023-2025-03-31.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2018-2023-12-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2015-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2019-2023-12-01.csv...
Processa

### Docentes

In [174]:
docentes_csvs = selecionar_csvs_capes('download_dir', 'docentes')

Buscando arquivos no diretório: capes_csv_files/docentes


In [175]:
docentes_csvs

['capes_csv_files/docentes/br-capes-colsucup-docente-2023-2025-03-31.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2013-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2019-2021-11-10.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2022-2025-03-31.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2015-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2020-2021-11-10.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2017-2021-11-10.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2021-2025-03-31.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2016-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2014-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2018-2021-11-10.csv']

In [176]:
processar_csvs(docentes_csvs, ['AN_BASE', 'ID_PESSOA', 'CD_AREA_AVALIACAO'], nome_arquivo_saida='docentes.csv')

Adicionando a coluna de filtro 'SG_ENTIDADE_ENSINO' a 'colunas_desejadas' para otimização da leitura.
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2023-2025-03-31.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2013-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2019-2021-11-10.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2022-2025-03-31.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2015-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2020-2021-11-10.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2017-2021-11-10.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2021-2025-03-31.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2016-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2014-2023-08-01.csv...
Processando capes_csv_files/docentes/b

### Programas

In [177]:
programas_csvs = selecionar_csvs_capes('download_dir', 'programas', padroes_regex=[r"br-capes-colsucup-prog"])

Buscando arquivos no diretório: capes_csv_files/programas


In [178]:
programas_csvs

['capes_csv_files/programas/br-capes-colsucup-prog-2017-2021-11-10.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2013.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2019-2021-11-10.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2022-2025-03-31.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2021-2025-03-31.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2023-2025-03-31.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2020-2021-11-10.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2018-2021-11-10.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2015.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2016.csv',
 'capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2014.csv']

In [180]:
processar_csvs(programas_csvs, ['AN_BASE', 'CD_PROGRAMA_IES', 'CD_AREA_AVALIACAO'], nome_arquivo_saida='programas.csv')

Adicionando a coluna de filtro 'SG_ENTIDADE_ENSINO' a 'colunas_desejadas' para otimização da leitura.
Processando capes_csv_files/programas/br-capes-colsucup-prog-2017-2021-11-10.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2013.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2019-2021-11-10.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2022-2025-03-31.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2021-2025-03-31.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2023-2025-03-31.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2020-2021-11-10.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2018-2021-11-10.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2015.csv...
Processando capes_csv_files/programas/br-capes-colsucup-prog-2013a2016-2020-06-12_2016.csv...
Processando capes_csv_files/

### Produção (artigos de periódicos por autor)

In [185]:
producao_csvs = selecionar_csvs_capes('download_dir', 'producao', padroes_regex=[r"br-capes-colsucup-prod-autor-.*bibliografica-artpe"])

Buscando arquivos no diretório: capes_csv_files/producao


In [186]:
producao_csvs

['capes_csv_files/producao/br-capes-colsucup-prod-autor-2021a2024-2025-03-31-bibliografica-artpe-2022.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2013a2016-2017-03-01-bibliografica-artpe.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2019.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2021a2024-2025-03-31-bibliografica-artpe-2023.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2021a2024-2025-03-31-bibliografica-artpe-2021.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2018.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2020.csv',
 'capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2017.csv']

In [188]:
processar_csvs(producao_csvs, ['AN_BASE', 'ID_ADD_PRODUCAO_INTELECTUAL', 'ID_PESSOA_DOCENTE', 'ID_PESSOA_DISCENTE'], nome_arquivo_saida='producao.csv')

Adicionando a coluna de filtro 'SG_ENTIDADE_ENSINO' a 'colunas_desejadas' para otimização da leitura.
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2021a2024-2025-03-31-bibliografica-artpe-2022.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2013a2016-2017-03-01-bibliografica-artpe.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2019.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2021a2024-2025-03-31-bibliografica-artpe-2023.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2021a2024-2025-03-31-bibliografica-artpe-2021.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2018.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2017a2020-2022-05-31-bibliografica-artpe-2020.csv...
Processando capes_csv_files/producao/br-capes-colsucup-prod-autor-2017